In [6]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests
from io import BytesIO

# url = 'https://www.shutterstock.com/image-vector/hello-world-handwritten-cursive-typography-260nw-1869117610.jpg'
# url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSMw98cNId3ybxaqKMg5NWTCSwAa-l8QHvrnhnNJzgtvjo1sfq3Av91zcq8xOYZSiWXqg&usqp=CAU'
# url = 'https://t3.ftcdn.net/jpg/03/61/93/62/360_F_361936252_rDwYjxmAPFe6mvPhwwvVGdSKgAbu30Xk.jpg'
# url = "https://images.squarespace-cdn.com/content/v1/599c5605b8a79bbe6db64021/1601043762216-KIGV2ZV6ONOLX7BT3POH/Screen+Shot+2020-09-25+at+10.16.38+AM.png"
url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRM-w7ZrDdpVNic2LoKHpZjugDvyl0ckVFBWQ&s" # bonjour
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# img.show()

image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_ids = model.generate(pixel_values, max_new_tokens=50)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("generated_text: ", generated_text)

# generated text를 언어인식해서 번역
from transformers import pipeline

# Load a language detection pipeline
lang_detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

# Detect the language
result = lang_detector(generated_text)
detected_lang = result[0]['label']
print('detected_lang: ', detected_lang)

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
from diffusers import FluxPipeline

# 번역 모델 불러오기
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
# 번역 모델에서 사용할 토크나이저 가져오기
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# 전처리
# Translate other language to Korean
tokenizer.src_lang = detected_lang + "_" + detected_lang.upper()
if (detected_lang == 'en' or detected_lang == 'es' or detected_lang == 'fr' or detected_lang == 'ja' or detected_lang == 'pt'):
    tokenizer.src_lang = detected_lang + "_XX"

# 토크나이징
encoded_lang = tokenizer(generated_text, return_tensors="pt")

# 추론
generated_tokens = model.generate(
    **encoded_lang,
    forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"]
)

# 후처리
# 번역 결과 decoding(자연어로 변경)
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print('translated_text', translated_text)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generated_text:  bonjour
detected_lang fr
tensor([[     2, 250014, 107687,      5,      2]])
['안녕하세요.']
